In [10]:
# import libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re
import random
import nltk
from scipy import sparse
from scipy.sparse import csr_matrix, vstack
from textblob import TextBlob
from langdetect import detect_langs
import pickle
from datetime import datetime
import string

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [11]:
# import data

# import glob
# import os

# path = r'/thaisongs' # use your path
# all_files = glob.glob(os.path.join(path , "/*.csv"))

all_files = ['./thaisongs/bodyslam.csv', './thaisongs/nont_tanont.csv', './thaisongs/tilly_birds.csv']

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

song_df = pd.concat(li, axis=0, ignore_index=True)

In [12]:
lyric_in_round_brackets = sum(list(song_df['lyric'].map(lambda s: re.findall(r'\((.*?)\)',s))), [])
print('Number of round brackets: {}'.format(len(lyric_in_round_brackets)))

lyric_in_square_brackets = sum(list(song_df['lyric'].map(lambda s: re.findall(r'\[(.*?)\]',s))), [])
print('Number of square brackets: {}'.format(len(lyric_in_square_brackets)))

lyric_in_curly_brackets = sum(list(song_df['lyric'].map(lambda s: re.findall(r'\{(.*?)\}',s))), [])
print('Number of curly brackets: {}'.format(len(lyric_in_curly_brackets)))

Number of round brackets: 180
Number of square brackets: 0
Number of curly brackets: 0


In [13]:
# remove round brackets but not text within
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r'\(|\)', '', s))

lyric_in_round_brackets = sum(list(song_df['lyric'].map(lambda s: re.findall(r'\((.*?)\)',s))), [])
print('Number of round brackets: {}'.format(len(lyric_in_round_brackets)))

Number of round brackets: 0


In [14]:
dot = sum(list(song_df['lyric'].map(lambda s: re.findall('\.',s))), [])
print('Number of dot: {}'.format(len(dot)))

Number of dot: 82


In [15]:
# remove dot 
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r'\.', '', s))

dot = sum(list(song_df['lyric'].map(lambda s: re.findall('\.',s))), [])
print('Number of dot: {}'.format(len(dot)))

Number of dot: 0


In [16]:
# count number of lines
song_df['lines'] = song_df['lyric'].map(lambda t: len(re.findall(r'\n', t)))

In [17]:
# remove line breaks
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r' \n|\n', '', s))

In [18]:
lyric_in_round_brackets = sum(list(song_df['lyric'].map(lambda s: re.findall(r'\((.*?)\)',s))), [])
print('Number of round brackets: {}'.format(len(lyric_in_round_brackets)))
song_df.head()

Number of round brackets: 0


,song_name,href,lyric,artist,lines
0,อะ ไนท์ (A Night),/music/thailyric/2681,ดึกดื่นคืนใด มองฟ้าไม่เห็นจันทร์ ไม่เจอะเจอกัน...,bodyslam,15
1,ขอบฟ้า,/music/thailyric/2640,ได้แต่เหลียวมองไปสุดฟ้าไกล\rมันช่างน่าเสียดายท...,bodyslam,29
2,คนที่ถูกรัก,/music/thailyric/2641,ใจ ฉันเคยถูกทิ้งเพียงลำพัง รักใครมาแล้วตั้งกี่...,bodyslam,25
3,ความเชื่อ,/music/thailyric/2644,มันเกือบจะล้มมันเหนื่อยมันล้าเหมือนแทบขาดใจ\rเ...,bodyslam,45
4,ความรักทำให้คนตาบอด,/music/thailyric/2647,รู้ว่ามันไม่สมควร ที่ยังกวนใจเธอ ฉันขอโทษที\rร...,bodyslam,21


Tokenise

In [23]:
import pythainlp
from pythainlp import word_tokenize
from pythainlp.corpus.common import thai_stopwords
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words

In [65]:
# from thainlplib import ThaiWordSegmentLabeller

In [24]:
import nltk
nltk.download('words')
th_stop = tuple(thai_stopwords())
en_stop = tuple(get_stop_words('en'))
p_stemmer = PorterStemmer()

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\jinnie\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


In [25]:
def split_word(text):
            
    
    tokens = word_tokenize(text,engine='newmm')
    
    # Remove stop words ภาษาไทย และภาษาอังกฤษ
    tokens = [i for i in tokens if not i in th_stop and not i in en_stop]
    
    # หารากศัพท์ภาษาไทย และภาษาอังกฤษ
    # English
    tokens = [p_stemmer.stem(i) for i in tokens]
    
    # Thai
    tokens_temp=[]
    for i in tokens:
        w_syn = wordnet.synsets(i)
        if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
            tokens_temp.append(w_syn[0].lemma_names('tha')[0])
        else:
            tokens_temp.append(i)
    
    tokens = tokens_temp
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

In [26]:
song_df['words'] = song_df['lyric'].map(lambda s: split_word(s))

In [27]:
song_df[['lyric','words']].tail()

,lyric,words
125,เจ็บที่ตรงไหน\rบอกฉันได้ไหม\rให้ฉันช่วยดู\r\rแ...,"[ปวด, ตรงไหน, \r, ไหม, \r, ดู, \r\r, รู้, \r, ..."
126,ฟ้า และท้องทะเลกับลมนั้นมีอยู่\rแต่ใครจะรู้ จะ...,"[ฟ้า, ท้องทะเล, ลม, \r, รู้, รู้, ใจ, \r, ไว, ..."
127,ทะเลาะเสียงดัง ข้าวของก็พัง อย่างนี้ประจำ\rบาง...,"[ทะเลาะ, ความดัง, สินค้า, พัง, ประจำ, \r, ช่วง..."
128,เธอทนอยู่กับเขามานาน อยู่กับเขามานาน\rฉันทนดูเ...,"[ทน, \r, ทน, ดู, ทรมาน, ไหว, \r, ยอมให้, เตือน..."
129,เจอมากี่ครั้งเรื่องรักๆ เดี๋ยวจะเล่าให้ฟัง\rชอ...,"[เจอ, กี่, เรื่อง, รัก, เดี๋ยว, เล่า, ฟัง, \r,..."


In [28]:
song_df['n_words'] = song_df['words'].map(len)

EDA

In [29]:
# number of songs
print('number of songs: ', str(len(song_df)))

# number of artists
print('number of artists: ', str(len(song_df['artist'].unique())))

number of songs:  130
number of artists:  3


In [30]:
# distribution songs per artist
song_count_df = song_df.groupby('artist')[['song_name']].count()
fig = px.histogram(song_count_df, x='song_name', title='Songs per artist', labels={'song_name': 'Songs'})
fig.show()

In [31]:
# distribution words per song
fig = px.histogram(song_df, x='n_words', title='Words per song')
fig.show()

In [32]:
# # create dataframe with lists of artists
song_df['words_str'] = song_df['words'].map(lambda lst: ' '.join(lst))

# # map text to artists
# words_to_artist = {}
# for tp in song_df[['artist', 'words_str']].itertuples(index=False):
#     artist = tp[0]
#     words = tp[1]
#     if words in words_to_artist:
#         words_to_artist[words].append(artist)
#     else:
#         words_to_artist[words] = [artist]

# # insert list of artists to dataframe
# song_df['artists'] = song_df['words_str'].map(words_to_artist)
# song_df['duplicates'] = song_df['artists'].map(len) - 1

# # convert list of artists to set of artists
# song_df['artists'] = song_df['artists'].map(set)
# song_df['n_artists'] = song_df['artists'].map(len)

# # remove duplicate songs
# artist_text_df = song_df.drop_duplicates('words_str')

Feature['artist'].value_counts() engineering

In [33]:
song_df

,song_name,href,lyric,artist,lines,words,n_words,words_str
0,อะ ไนท์ (A Night),/music/thailyric/2681,ดึกดื่นคืนใด มองฟ้าไม่เห็นจันทร์ ไม่เจอะเจอกัน...,bodyslam,15,"[ดึกดื่น, คืน, ใด, ฟ้า, เดือน, เจอะ, เจอกัน, ร...",86,ดึกดื่น คืน ใด ฟ้า เดือน เจอะ เจอกัน รู้ หาย \...
1,ขอบฟ้า,/music/thailyric/2640,ได้แต่เหลียวมองไปสุดฟ้าไกล\rมันช่างน่าเสียดายท...,bodyslam,29,"[เหลียว, ฟ้า, \r, ช่าง, น่าเสียดาย, ขอบฟ้า, \r...",92,เหลียว ฟ้า \r ช่าง น่าเสียดาย ขอบฟ้า \r\rจบ เส...
2,คนที่ถูกรัก,/music/thailyric/2641,ใจ ฉันเคยถูกทิ้งเพียงลำพัง รักใครมาแล้วตั้งกี่...,bodyslam,25,"[ใจ, ทิ้ง, เพียงลำพัง, รัก, กี่, \r, ผิดหวัง, ...",105,ใจ ทิ้ง เพียงลำพัง รัก กี่ \r ผิดหวัง กี่ รัก ...
3,ความเชื่อ,/music/thailyric/2644,มันเกือบจะล้มมันเหนื่อยมันล้าเหมือนแทบขาดใจ\rเ...,bodyslam,45,"[ล้ม, เหนื่อย, ล้า, เหมือน, แทบ, ขาดใจ, \r, เด...",163,ล้ม เหนื่อย ล้า เหมือน แทบ ขาดใจ \r เดิน ท้อ เ...
4,ความรักทำให้คนตาบอด,/music/thailyric/2647,รู้ว่ามันไม่สมควร ที่ยังกวนใจเธอ ฉันขอโทษที\rร...,bodyslam,21,"[รู้, สมควร, ยั่ว, ขอโทษ, \r, รัก, คอย, หวังดี...",65,รู้ สมควร ยั่ว ขอโทษ \r รัก คอย หวังดี มีสิทธิ...
...,...,...,...,...,...,...,...,...
125,ให้เธอหายดี (SuperMom),/music/thailyric/18442,เจ็บที่ตรงไหน\rบอกฉันได้ไหม\rให้ฉันช่วยดู\r\rแ...,tilly_birds,50,"[ปวด, ตรงไหน, \r, ไหม, \r, ดู, \r\r, รู้, \r, ...",91,ปวด ตรงไหน \r ไหม \r ดู \r\r รู้ \r \r \r\r ปว...
126,ให้เธอ,/music/thailyric/19409,ฟ้า และท้องทะเลกับลมนั้นมีอยู่\rแต่ใครจะรู้ จะ...,tilly_birds,51,"[ฟ้า, ท้องทะเล, ลม, \r, รู้, รู้, ใจ, \r, ไว, ...",122,ฟ้า ท้องทะเล ลม \r รู้ รู้ ใจ \r ไว ั \r ตะวัน...
127,อภัย (Broken),/music/thailyric/15735,ทะเลาะเสียงดัง ข้าวของก็พัง อย่างนี้ประจำ\rบาง...,tilly_birds,31,"[ทะเลาะ, ความดัง, สินค้า, พัง, ประจำ, \r, ช่วง...",99,ทะเลาะ ความดัง สินค้า พัง ประจำ \r ช่วงเวลา เส...
128,อย่าอยู่เลย,/music/thailyric/19971,เธอทนอยู่กับเขามานาน อยู่กับเขามานาน\rฉันทนดูเ...,tilly_birds,46,"[ทน, \r, ทน, ดู, ทรมาน, ไหว, \r, ยอมให้, เตือน...",170,ทน \r ทน ดู ทรมาน ไหว \r ยอมให้ เตือน กี่ ฟัง ...


## Feature engineering

### Number of words

In [34]:
n_artist = 4
random.seed(0)

artist_select = random.choices(song_df['artist'].unique(), k=n_artist)

song_filter_df = song_df.loc[song_df['artist'].isin(artist_select)]
print('Total number of songs: {}'.format(len(song_filter_df)))
song_filter_df.groupby('artist')[['song_name']].count().reset_index().rename(columns={'song':'songs'})

Total number of songs: 130


,artist,song_name
0,bodyslam,66
1,nont_tanont,36
2,tilly_birds,28


In [35]:
fig = px.box(song_filter_df, x='artist', y='n_words', title='Word count per song by artist')
fig.show()

### Repeated words['unique_words_ratio'])song_filter_df.join(

In [36]:
# number of unique stems
song_df['n_unique_words'] = song_df['words'].map(lambda lst: len(set(lst)))

# ratio of unique stems
song_df['unique_words_ratio'] = song_df['n_unique_words'] / song_df['n_words']

# attach column to selected artists
song_filter_df = song_filter_df.join(song_df['unique_words_ratio'])

In [37]:
fig = px.box(song_filter_df, x='artist', y='unique_words_ratio', title='Ratio of unique words to all words')
fig.show()

### Words per line

In [38]:
# calculate number of words per line
song_df['words_per_line'] = song_df['n_words'] / song_df['lines'].astype(float)

song_filter_df = song_filter_df.join(song_df['words_per_line'])

In [39]:
fig = px.box(song_filter_df, x='artist', y='words_per_line', title='Words per line')
fig.show()

## TFIDF

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [55]:
# initialise count vectorizer
cv = CountVectorizer(analyzer=lambda x:x.split())

word_count_vector = cv.fit_transform(song_df['words_str'])

# compute idf
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [56]:
# print idf values
tfidf_df = pd.DataFrame({'word': cv.get_feature_names(), 'weight': tfidf_transformer.idf_})
 
# get lowest weights
tfidf_df.sort_values('weight').head()

,word,weight
1729,ใจ,1.364338
968,รู้,1.444381
179,คน,1.493171
943,รัก,1.505749
1262,หัวใจ,1.531392


In [57]:
cv.get_feature_names()

["'",
 ',',
 '/',
 ':',
 'Ah',
 'I',
 'It',
 'No',
 'Oh',
 '\\',
 'ah',
 'all',
 'alon',
 'app',
 'arin',
 'ask',
 'audit',
 'babepoom',
 'babi',
 'brand',
 'call',
 'can',
 'chamil',
 'come',
 'comment',
 'complet',
 'continu',
 'crazi',
 'day',
 'die',
 'everi',
 'everytim',
 'fall',
 'feel',
 'friend',
 'gene',
 'get',
 'girl',
 'goin',
 'good',
 'goodby',
 'goodnight',
 'heart',
 'hmmm',
 'huh',
 'just',
 'know',
 'lab',
 'like',
 'listen',
 'll',
 'love',
 'ma',
 'make',
 'miss',
 'need',
 'never',
 'new',
 'night',
 'oh',
 'ohh',
 'ooh',
 'play',
 'real',
 'rock',
 'say',
 'sequel',
 'sister',
 'soon',
 'star',
 'start',
 'thousand',
 'time',
 'vers',
 'who',
 'whole',
 'will',
 'wo',
 'woo',
 'wooo',
 'word',
 'world',
 'yeah',
 'you',
 'ก',
 'กฎ',
 'กฎแห่งกรรม',
 'กด',
 'กรอบรูป',
 'กระจก',
 'กระต่าย',
 'กระทะ',
 'กรัก',
 'กราว',
 'กรีด',
 'กร่อน',
 'กร้าน',
 'กลม',
 'กลมกล่อม',
 'กลับคืน',
 'กลับมา',
 'กลัว',
 'กลาง',
 'กลาย',
 'กลิ้ง',
 'กลืน',
 'กวน',
 'กว่าเพื่อน',
 'กว้างใ

In [58]:
# get highest weights
tfidf_df.sort_values('weight', ascending=False).head()

,word,weight
1801,﻿,5.18205
853,มัวหมอง,5.18205
874,มีด,5.18205
1594,เเนบ,5.18205
871,มีคุณค่า,5.18205


In [60]:
# assign tf idf scores to each song
tf_idf_vector = tfidf_transformer.transform(word_count_vector)

# attach count vectors to dataframe
tf_idf_vector_lst = [-1] * len(song_df)
for i in range(len(song_df)):
    tf_idf_vector_lst[i] = tf_idf_vector[i]
song_df['tf_idf_vector'] = tf_idf_vector_lst    

song_df['tf_idf_score'] = song_df['tf_idf_vector'].map(lambda vec: np.sum(vec.todense()))

# join valus to selected artists
song_filter_df = song_filter_df.join(song_df[['tf_idf_vector', 'tf_idf_score']])

ValueError: columns overlap but no suffix specified: Index(['tf_idf_vector', 'tf_idf_score'], dtype='object')

In [91]:
t_feat2.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [61]:
fig = px.box(song_filter_df, x='artist', y='tf_idf_score', title='TFIDF scores of songs per artist')
fig.show()

In [62]:
# calculate mean vector
def get_mean_vector(vec_lst):
    return csr_matrix(vstack(vec_lst).mean(axis=0))

In [63]:
# calculate mean vector over all songs of same artist
artist_df = song_df.groupby('artist').agg({'tf_idf_vector': get_mean_vector, 'song_name': len}).reset_index()\
                   .rename(columns={'song_name': 'songs'})

# get selected artists
artist_filter_df = artist_df.loc[artist_df['artist'].isin(song_filter_df['artist'])]

In [65]:
similarity_matrix = cosine_similarity(vstack(artist_filter_df['tf_idf_vector']), 
                                      vstack(artist_filter_df['tf_idf_vector']))
artist_names = artist_filter_df['artist'].tolist()
fig = go.Figure(data=go.Heatmap(z=np.flipud(similarity_matrix), x=artist_names, y=list(reversed(artist_names)), 
                                colorscale='balance', zmin=0.5, zmax=1.1))
fig.show()